# Stokes Gleichung (2D)

## Fallender Block (konstante Viskosität, zeitabhängig)

Beschäftigen wir uns nun mit dem **zeitabhängigen**, isoviskosen  Problem des fallenden Blocks. 

Dazu müssen wir die Lösung der **Impulserhaltung** mit der Lösung der **Advektionsgleichung** koppeln. Die Koppelung führen wir durch, indem wir zuerst die Impulserhaltung nach den Geschwindigkeiten lösen und dann die Dichte (oder im Falle von passiven Markern, die Phase) mit der Advektionsgleichung transportieren. 

### Das Problem

Anders als im vollständigen Benchmark für dieses Problem, konzentrieren wir uns hier nur auf den **isoviskosen** Fall (im Benchmark betrachtet man die Sinkgeschwindigkeit und Deformation des Blockes in Abhängigkeit des Viskositätskontrastes [siehe hier](../../examples/Benchmarks/)). 

Dabei nehmen wir einen quadratischen Körper mit einer gewissen Breite (**W**), Höhe (**T**), und Dichte ($\rho_b$) an, der sich in einem bestimmten viskosen Medium ($\eta_m$) mit einer gewissen Dichte ($\rho_m$) befindet. Die zu modellierende Umgebung ist ebenfalls quadratisch mit einer gewissen Länge ($L=500\ \textrm{km}$) und Höhe ($H=500\ \textrm{km}$) und wir nehmen überall *free slip* Geschwindigkeitsrandbedingungen an.  

<img src="../Figures/Exercise09_1.png" alt="drawing" width="450"/> <br>
**Abb. 1.** Modelaufbau 

Eine detaillierte Beschreibung wie die Modelldomäne durch ein numerisches Gitter aufgeteilt werden kann, sowie die Diskretisierung der **Impulserhaltung** und **Massenerhaltung** in Abhängigkeit von *free slip* oder *no slip* Randbedingungen ist [hier](../../examples/StokesEquation/2D/README.md) gegeben. Auf Grund des Umfanges verzichten wir hier auf eine detaillierte Erklärung. 

Wir würden gerne möglichst viele der Advektionsmechanismen in das Problem einbauen, so dass wir neben der Änderung für die Zeitabhängigkeit noch zusätzliche Änderungen vornehmen müssen. Die Änderungen werden im Weiteren in **fett**-gedruckter Schrift gekennzeichnet. 

Für zusätzliche Informationen siehe die [vorrangehende Übung](09_2D_Falling_Block.ipynb).

Laden wir zuerst einmal die notwendigen Module zur Visualisierung der Ergebnisse, Lösen des linearen Gleichungssystems, Aufstellen der Anfangsbedingungen, und dem Lösen der zweidimensionalen Impulserhaltuing. Durch die Verwendung der Advektion und der Tracer kommen noch zusätzliche Module hinzu. 

In [1]:
using Plots
using ExtendableSparse
using GeoModBox.InitialCondition, GeoModBox.MomentumEquation.TwoD
using GeoModBox.AdvectionEquation.TwoD
using GeoModBox.Tracers.TwoD
using Base.Threads
using Printf

Zuerst definieren wir mit Hilfe des **Tupels FD die Methode der Advektion**.

In [2]:
# Define Numerical Scheme =========================================== #
# Advection ---
#   1) upwind, 2) slf, 3) semilag, 4) tracers
FD          =   (Method     = (Adv=:tracers,),)
# ------------------------------------------------------------------- #

(Method = (Adv = :tracers,),)

Mit Hilfe der eingebauten Funktion ```IniPhase()``` lässt sich die Anfangsdichteverteilung für unser Problem *block* erstellen. Je nach Bedarf, erstellt die Funktion eine Anfangsverteilung von bestimmten Größen auf dem numerischen Gitter. Zur Nutzung der Funktion brauchen wir das Tuple ```Ini```, in dem die Verteilung definiert ist (hier für die Phase *p*). Die mit ```IniPhase()``` erstellte Dichteverteilung auf den Zentroids wird von allen Advektionsmethoden verwendet, außer im Falle der Tracer. Für diese wird eine alternative Initialisierung angewendet. 

In [3]:
# Define Initial Condition ========================================== #
# Density --- 
#   1) block
Ini         =   (p=:block,) 
# ------------------------------------------------------------------- #
# Plot Settings ===================================================== #
Pl  =   (
    qinc    =   5,
    qsc     =   100*(60*60*24*365.25)*1e0
)
# ------------------------------------------------------------------- #

(qinc = 5, qsc = 3.15576e9)

Als nächstes definieren wir die Geometrie unsere Modeldomäne. 

In [4]:
 # Geometry ========================================================== #
 M       =   (
    xmin    =   0.0,
    xmax    =   500.0e3,    # [ m ]
    ymin    =   -500.0e3,   # [ m ]
    ymax    =   0.0,
)
# ------------------------------------------------------------------- #

(xmin = 0.0, xmax = 500000.0, ymin = -500000.0, ymax = 0.0)

Nun definieren wir unsere Gitter Auflösung $\left(nc_x = nc_y = 50\right)$ und das numerische Gitter.

Dabei müssen wir sowohl die Gitterbreite $\Delta{x}$ und $\Delta{y}$ als auch die Koordinaten der unterschiedlichen Gitter (*Eckpunkte* und *Zentroids*) angeben. Dazu definieren wir zuerst die jeweiligen 1-D Koordinatenvektoren, aus denen dann die 2-D Koordinatengitter erstellt werden.  

In [5]:
# Grid ============================================================== #
NC      =   (
    x   =   50, 
    y   =   50,
)
NV      =   (
    x   =   NC.x + 1,
    y   =   NC.y + 1,
)
Δ       =   (
    x   =   (M.xmax - M.xmin)/NC.x,
    y   =   (M.ymax - M.ymin)/NC.y,
)
x       =   (
    c   =   LinRange(M.xmin+Δ.x/2,M.xmax-Δ.x/2,NC.x),
    ce  =   LinRange(M.xmin - Δ.x/2.0, M.xmax + Δ.x/2.0, NC.x+2),
    v   =   LinRange(M.xmin,M.xmax,NV.x),
)
y       =   (
    c   =   LinRange(M.ymin+Δ.y/2,M.ymax-Δ.y/2,NC.y),
    ce  =   LinRange(M.ymin - Δ.x/2.0, M.ymax + Δ.x/2.0, NC.y+2),
    v   =   LinRange(M.ymin,M.ymax,NV.y),
)
x1      =   (
    c2d     =   x.c .+ 0*y.c',
    v2d     =   x.v .+ 0*y.v', 
    vx2d    =   x.v .+ 0*y.ce',
    vy2d    =   x.ce .+ 0*y.v',
)
x   =   merge(x,x1)
y1      =   (
    c2d     =   0*x.c .+ y.c',
    v2d     =   0*x.v .+ y.v',
    vx2d    =   0*x.v .+ y.ce',
    vy2d    =   0*x.ce .+ y.v',
)
y   =   merge(y,y1)
# ------------------------------------------------------------------- #

(c = LinRange{Float64}(-495000.0, -5000.0, 50), ce = LinRange{Float64}(-505000.0, 5000.0, 52), v = LinRange{Float64}(-500000.0, 0.0, 51), c2d = [-495000.0 -484999.99999999994 … -15000.000000000011 -5000.0; -495000.0 -484999.99999999994 … -15000.000000000011 -5000.0; … ; -495000.0 -484999.99999999994 … -15000.000000000011 -5000.0; -495000.0 -484999.99999999994 … -15000.000000000011 -5000.0], v2d = [-500000.0 -490000.0 … -10000.00000000001 0.0; -500000.0 -490000.0 … -10000.00000000001 0.0; … ; -500000.0 -490000.0 … -10000.00000000001 0.0; -500000.0 -490000.0 … -10000.00000000001 0.0], vx2d = [-505000.0 -495000.0 … -5000.00000000002 5000.0; -505000.0 -495000.0 … -5000.00000000002 5000.0; … ; -505000.0 -495000.0 … -5000.00000000002 5000.0; -505000.0 -495000.0 … -5000.00000000002 5000.0], vy2d = [-500000.0 -490000.0 … -10000.00000000001 0.0; -500000.0 -490000.0 … -10000.00000000001 0.0; … ; -500000.0 -490000.0 … -10000.00000000001 0.0; -500000.0 -490000.0 … -10000.00000000001 0.0])

Nun müssen wir die physikalischen Parameter definieren, die Graviationsbeschleunigung $g$, die Viskosität $\eta_0$ (10<sup>21</sup> Pa s), die Hintergunddichte $\rho_0$ (3200 kg/m<sup>3</sup>), und die Dichte des Blocks $\rho_1$ (3300 kg/m<sup>3</sup>).

Der Vektor ```phase``` beinhaltet die Phasennummer, welche in der Funktion ```IniPhase()``` und bei der Initialisierung der Marker (```IniTracer2D()```) verwendet wird um die jeweilge Phase zuzuordnen. Der Vektor $\rho$ beinhaltet die Dichtewerte die der jeweilige Phase zugeordnet sind. 

In [6]:
 # Physics =========================================================== #
 g       =   9.81

 η₀      =   1.0e21

 ρ₀      =   3200.0          #   Background density
 ρ₁      =   3300.0          #   Block density
 ρ       =   [ρ₀,ρ₁] 

 phase   =   [0,1]
 # ------------------------------------------------------------------- #

2-element Vector{Int64}:
 0
 1

**Für unser zeitabhängiges Problem defenieren wir nur wieder den Ort zum speichern der Animation.**

In [7]:
# Animationsettings ================================================= #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("10_Falling_",Ini.p,"_iso_",FD.Method.Adv)
save_fig    =   1
# ------------------------------------------------------------------- #

1

Nun legen wir die Größen der benötigten Felder fest. **Dabei verwenden wir einige neue Felder.** 

In [8]:
 # Allocation ======================================================== #
 D   =   (
    vx      =   zeros(Float64,NV.x,NC.y+2),
    vy      =   zeros(Float64,NC.x+2,NV.y),
    Pt      =   zeros(Float64,NC...),
    p       =   zeros(Float64,NC...),
    p_ex    =   zeros(Float64,NC.x+2,NC.y+2),
    ρ       =   zeros(Float64,NC...),
    ρ_ex    =   zeros(Float64,NC.x+2,NC.y+2),
    ρ_exo   =   zeros(Float64,NC.x+2,NC.y+2),
    vxc     =   zeros(Float64,NC...),
    vyc     =   zeros(Float64,NC...),
    vc      =   zeros(Float64,NC...),
    wt      =   zeros(Float64,(NC.x,NC.y)),
)
# ------------------------------------------------------------------- #

(vx = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], vy = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Pt = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], p = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], p_ex = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], ρ = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], ρ_ex = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], ρ_exo = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], vxc = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], vyc = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], vc = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], wt = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 

Jetz lassen sich die Randbedingungen und die Anfangsbedingungen festlegen. 

Für die Randbedingungen benutzen wir wieder das Tuple ```VBC```, in dem die Art des Randes (*free slip* oder *no slip*) an der jeweiligen Position (**E**, **W**, **S**,**N**) definiert ist. Der angegeneben Wert ```val``` bestimmt dann den Wert der Geschwindigkeit an dem jeweiligen Rand. 

In [9]:
# Boundary Conditions =============================================== #
VBC     =   (
    type    =   (E=:freeslip,W=:freeslip,S=:freeslip,N=:freeslip),
    val     =   (E=zeros(NV.y),W=zeros(NV.y),S=zeros(NV.x),N=zeros(NV.x)),
)
# ------------------------------------------------------------------- #

(type = (E = :freeslip, W = :freeslip, S = :freeslip, N = :freeslip), val = (E = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], W = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], S = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], N = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))

**Als nächstes müssen wir die Parameter für die Zeit und die Zeitschrittlänge definieren.**

In [10]:
# Time ============================================================== #
T   =   ( 
    tmax    =   [0.0],  
    Δfac    =   1.0,    # Courant time factor, i.e. dtfac*dt_courant
    Δ       =   [0.0],
    time    =   [0.0,0.0],
)
T.tmax[1]   =   9.886 * 1e6 * (60*60*24*365.25)   # [ s ]
nt          =   9999
# ------------------------------------------------------------------- #

9999

Zum Verwenden der **passiven Tracers zur Advektion der Dichte müssen diese ersteinmal initialisiert werden**. Die Funktion ```IniTracer2D()``` initialisiert die Verteilung der Tracer in der gegebenen Modeldomäne (mit beliebiger Störung bei der Position) und, falls vorher in ```Ini``` definiert, wird den Tracers die jeweilige Phase zugewiesen. 

Die Tracer werden im der Zeitschleife advektiert und transportieren die Phase. Mit der Funktion ```Marker2Cells()``` wird mit Hilfe der jeweiligen Phase eine Eigenschaft der Phase auf das Gitter interpoliert (hier die Dichte $\rho$).

Falls eine andere Advektionmethode gewählt wird, wird die Dichte direkt auf dem Gitter initialisiert. Beachte, die Advetkionsroutinen benötigen die Information auf dem erweiterten Gitter. 

Zum Aufstellen der Anfangsbedingungen, verwenden wir die Funktion ```IniPhase``` in der dem jeweiligen Punkt die dazugehörige Phase zugeteilt wird. Darauf weisen wir der jeweilgen Phase den entsprechenden Dichte Wert hinzu. 

In [11]:
# Tracer Advection ================================================== #
if FD.Method.Adv==:tracers 
    # Tracer Initialization ---
    nmx,nmy     =   3,3
    noise       =   1
    nmark       =   nmx*nmy*NC.x*NC.y
    Aparam      =   :phase
    MPC         =   (
        c               =   zeros(Float64,(NC.x,NC.y)),
        th              =   zeros(Float64,(nthreads(),NC.x,NC.y)),                
        min             =   zeros(Float64,nt),
        max             =   zeros(Float64,nt),
        mean            =   zeros(Float64,nt),
    )
    MPC1        = (
        PG_th   =   [similar(D.ρ) for _ = 1:nthreads()], # per thread
        wt_th   =   [similar(D.wt) for _ = 1:nthreads()], # per thread
    )
    MPC     =   merge(MPC,MPC1)
    Ma      =   IniTracer2D(Aparam,nmx,nmy,Δ,M,NC,noise,Ini.p,phase)
    # RK4 weights ---
    rkw     =   1.0/6.0*[1.0 2.0 2.0 1.0]   # for averaging
    rkv     =   1.0/2.0*[1.0 1.0 2.0 2.0]   # for time stepping
    # Count marker per cell ---
    CountMPC(Ma,nmark,MPC,M,x,y,Δ,NC,1)
    # Interpolate from markers to cell ---
    Markers2Cells(Ma,nmark,MPC.PG_th,D.ρ,MPC.wt_th,D.wt,x,y,Δ,Aparam,ρ)
    # D.ρ_ex[2:end-1,2:end-1]     .= D.ρ
else
    # --------------------------------------------------------------- #
    # Initial Condition ============================================= #
    # Phase ---
    # If tracers are used, phases need to be defined on the tracers 
    # directly and are not suppose to be interpolated from the centroids! 
    IniPhase!(Ini.p,D,M,x,y,NC;phase)
    for i in eachindex(phase)
        D.ρ[D.p.==phase[i]] .= ρ[i]
    end
    D.ρ_ex[2:end-1,2:end-1]     .=  D.ρ
    D.ρ_ex[1,:]     .=   D.ρ_ex[2,:]
    D.ρ_ex[end,:]   .=   D.ρ_ex[end-1,:]
    D.ρ_ex[:,1]     .=   D.ρ_ex[:,2]
    D.ρ_ex[:,end]   .=   D.ρ_ex[:,end-1]
end
# ------------------------------------------------------------------- #

0 markers out


Nun müssen die Parameter für das lineare Gleichngssstem erstellt werden. 

Die Nummerierung der Gleichungen für die *x-Komponente* und die *y-Komnponente* der Impulserhaltung, sowie die der *Massenerhaltung*, als auch die Diskretisierung der zu lösenden Gleichungen mit Hilfe der finiten Differenzen Methode ist im Detail [hier](../../examples/StokesEquation/2D/README.md) beschrieben. 

Außerdem muss noch der Vektor der rechten Seite ```rhs```, sowie der Lösungsvektor ```χ``` initialisiert werden. Die Größe von beiden ist durch die maximale Anzahl der Gleichungen gegeben. 

In [12]:
# System of Equations =============================================== #
# Numbering, without ghost nodes! ---
off    = [  NV.x*NC.y,                          # vx
            NV.x*NC.y + NC.x*NV.y,              # vy
            NV.x*NC.y + NC.x*NV.y + NC.x*NC.y]  # Pt

Num    =    (
    Vx  =   reshape(1:NV.x*NC.y, NV.x, NC.y), 
    Vy  =   reshape(off[1]+1:off[1]+NC.x*NV.y, NC.x, NV.y), 
    Pt  =   reshape(off[2]+1:off[2]+NC.x*NC.y,NC...),
)
χ       =   zeros(maximum(Num.Pt))  #   Unknown Vector
rhs     =   zeros(maximum(Num.Pt))  #   Right-hand Side
# ------------------------------------------------------------------- #

7600-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

Mit Hilfe der Funktion ```Assemblyc()``` lassen sich die Koeffizienten für die Koeffizientenmatrix $\bold{K}$ zuweisen. Die Funktion weist der jeweiligen Gleichung den Wert der benötigten Koeffizienten zu. Da wir ein isoviskoses Problem betrachten, können wir die Koeffizientenmatrix einmal außerhalb der Zeitschleife aufstellen, da sie sich nicht ändert. 

In [13]:
# Assemble Coefficients ============================================= #
K       =   Assemblyc(NC, NV, Δ, η₀, VBC, Num)
# ------------------------------------------------------------------- #

7600×7600 ExtendableSparseMatrixCSC{Float64, Int64} with 44304 stored entries:
⎡⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⢆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠦⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦⡀⎥
⎢⠦⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⎥
⎢⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎦

Nun folgt die Zeitschleife. In dieser muss jeweils 
- die rechte Seite aktualisiert (enthält die Dichteverteilung) 
- das Gleichungssystem gelöst
- die Dichteverteilung und Geschwindigkeit visualisiert
- die Zeitschrittlänge berechnet
- und die Dichte advektiert werden. 

In [14]:
# Time Loop ========================================================= #
for it = 1:nt
    # Update Time ---
    T.time[1]   =   T.time[2] 
    @printf("Time step: #%04d, Time [Myr]: %04e\n ",it,T.time[1]/(60*60*24*365.25)/1.0e6)
    # Momentum Equation ===
    # Update RHS ---
    rhs     =   updaterhsc( NC, NV, Δ, η₀, D.ρ, g, VBC, Num, rhs )
    # Solve System of Equations ---
    χ       =   K \ rhs
    # Update Unknown Variables ---
    D.vx[:,2:end-1]     .=  χ[Num.Vx]
    D.vy[2:end-1,:]     .=  χ[Num.Vy]
    D.Pt                .=  χ[Num.Pt]
    # ===
    # Get the velocity on the centroids ---
    for i = 1:NC.x
        for j = 1:NC.y
            D.vxc[i,j]  = (D.vx[i,j+1] + D.vx[i+1,j+1])/2
            D.vyc[i,j]  = (D.vy[i+1,j] + D.vy[i+1,j+1])/2
        end
    end
    @. D.vc        = sqrt(D.vxc^2 + D.vyc^2)

    @show(maximum(D.vc))
    @show(minimum(D.Pt))
    @show(maximum(D.Pt))

    if T.time[2] >= T.tmax[1]
        it = nt
    end

    if mod(it,2) == 0 || it == nt || it == 1
        p = heatmap(x.c./1e3,y.c./1e3,D.ρ',color=:inferno,
                xlabel="x[km]",ylabel="y[km]",colorbar=false,
                title="Density",
                aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3), 
                ylims=(M.ymin/1e3, M.ymax/1e3),
                layout=(2,2),subplot=1)
        quiver!(p,x.c2d[1:Pl.qinc:end,1:Pl.qinc:end]./1e3,
                    y.c2d[1:Pl.qinc:end,1:Pl.qinc:end]./1e3,
                    quiver=(D.vx[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc,
                            D.vyc[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc),        
                    color="white",layout=(2,2),subplot=1)
        heatmap!(p,x.c./1e3,y.c./1e3,D.vxc',
                    xlabel="x[km]",ylabel="y[km]",colorbar=false,
                    title="V_x",
                    aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
                    ylims=(M.ymin/1e3, M.ymax/1e3),
                    layout=(2,2),subplot=3)
        heatmap!(p,x.c./1e3,y.c./1e3,D.vyc',
                    xlabel="x[km]",ylabel="y[km]",colorbar=false,
                    title="V_y",
                    aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
                    ylims=(M.ymin/1e3, M.ymax/1e3),
                    layout=(2,2),subplot=4)
        heatmap!(p,x.c./1e3,y.c./1e3,D.Pt',
                    xlabel="x[km]",ylabel="y[km]",colorbar=false,
                    title="P_t",
                    aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
                    ylims=(M.ymin/1e3, M.ymax/1e3),
                    layout=(2,2),subplot=2)
        if save_fig == 1
            Plots.frame(anim)
        elseif save_fig == 0
            display(p)
        end
    end
    if T.time[2] >= T.tmax[1]
        break
    end
     # Calculate Time Stepping ---
    T.Δ[1]      =   T.Δfac * minimum((Δ.x,Δ.y)) / 
                        (sqrt(maximum(abs.(D.vx))^2 + maximum(abs.(D.vy))^2))
    
    @show T.Δ[1]
    @printf("\n")
    # Calculate Time ---
    T.time[2]   =   T.time[1] + T.Δ[1]
    if T.time[2] > T.tmax[1] 
        T.Δ[1]      =   T.tmax[1] - T.time[1]
        T.time[2]   =   T.time[1] + T.Δ[1]
    end
    # Advection ===
    if FD.Method.Adv==:upwind
        upwindc2D!(D.ρ,D.ρ_ex,D.vxc,D.vyc,NC,T.Δ[1],Δ.x,Δ.y)
    elseif FD.Method.Adv==:slf
        slfc2D!(D.ρ,D.ρ_ex,D.ρ_exo,D.vxc,D.vyc,NC,T.Δ[1],Δ.x,Δ.y)
    elseif FD.Method.Adv==:semilag
        semilagc2D!(D.ρ,D.ρ_ex,D.vxc,D.vyc,[],[],x,y,T.Δ[1])
    elseif FD.Method.Adv==:tracers
        @printf("Running on %d thread(s)\n", nthreads())        
        # Advect tracers ---
        AdvectTracer2D(Ma,nmark,D,x,y,T.Δ[1],Δ,NC,rkw,rkv,1)
        CountMPC(Ma,nmark,MPC,M,x,y,Δ,NC,it)
      
        # Interpolate phase from tracers to grid ---
        Markers2Cells(Ma,nmark,MPC.PG_th,D.ρ,MPC.wt_th,D.wt,x,y,Δ,Aparam,ρ)
        # D.ρ_ex[2:end-1,2:end-1]     .= D.ρ
    end
end # End Time Loop

Time step: #0001, Time [Myr]: 0.000000e+00
 maximum(D.vc) = 8.313929890044719e-10
minimum(D.Pt) = -1.7771316137909174e9
maximum(D.Pt) = 1.3640010615802097e10
T.Δ[1] = 1.010047824601266e13

Running on 1 thread(s)
0 markers out
Time step: #0002, Time [Myr]: 3.200648e-01
 maximum(D.vc) = 8.676932090072663e-10
minimum(D.Pt) = -1.5530878369501755e9
maximum(D.Pt) = 1.3863383547322845e10
T.Δ[1] = 9.785081580372785e12

Running on 1 thread(s)
0 markers out
Time step: #0003, Time [Myr]: 6.301354e-01
 maximum(D.vc) = 8.763631546649434e-10
minimum(D.Pt) = -3.354734911953421e9
maximum(D.Pt) = 1.205948385997741e10
T.Δ[1] = 9.8632663453049e12

Running on 1 thread(s)
0 markers out
Time step: #0004, Time [Myr]: 9.426834e-01
 maximum(D.vc) = 9.004196065387808e-10
minimum(D.Pt) = -2.585906724977166e9
maximum(D.Pt) = 1.2827303807853706e10
T.Δ[1] = 9.714053326085236e12

Running on 1 thread(s)
0 markers out
Time step: #0005, Time [Myr]: 1.250503e+00
 maximum(D.vc) = 9.353966348467277e-10
minimum(D.Pt) = -2.

Zuletzt speichern wir noch die Animation. 

In [15]:
# Save Animation ==================================================== #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
    foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
end

┌ Info: Saved animation to /home/lufuchs/progs/src/julia/scripts/GeoModBox_p.jl/exercises/Correction/Results/10_Falling_block_iso_tracers.gif
└ @ Plots /home/lufuchs/.julia/packages/Plots/Ec1L1/src/animation.jl:156
